#**유통 판매량 예측 및 재고 최적화**
## **단계2 : 모델링을 위한 데이터 전처리**

# **0.미션**

* 단계1 상품별 데이터셋 만들기
    * 가설로 도출된 변수를 반영한 데이터셋을 상품별로 생성
* 단계2 기초 모델 생성
    * base line model : dense layer만 이용해서 기초 모델 생성
    * LSTM, CNN 알고리즘을 이용해서 기초 모델 생성

* 대상 상품(핵심 상품)

| Product_ID|Product_Code|SubCategory|Category|LeadTime|Price|
|----|----|----|----|----|----|
|3|DB001|Beverage|Drink|2|8|
|12|GA001|Milk|Food|3|6|
|42|FM001|Agricultural products|Grocery|3|5|


# **1.환경설정**

* 세부 요구사항
    * 구글콜랩
        * 구글 드라이브 바로 밑에 project 폴더를 만들고,
        * 데이터 파일을 복사해 넣습니다.
    - 라이브러리 로딩
        * 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.


### **(1) 경로 설정**

* 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/project6/'

### **(2) 라이브러리 설치 및 불러오기**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

from keras.models import Sequential
from keras.backend import clear_session
from keras.layers import Dense, LSTM, Flatten, Conv1D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings("ignore")

In [4]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.plot(history['loss'], label='train_err', marker = '.')
    plt.plot(history['val_loss'], label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

In [5]:
# 예측 결과 시각화
def plot_model_result(y_train, y_val, pred) :

    y_train = pd.Series(y_train)
    y_val = pd.Series(y_val)
    y_val.index = range(len(y_train), len(y_train) + len(y_val))

    pred = pd.Series(pred.reshape(-1,), index = y_val.index)

    # 전체 시각화
    plt.figure(figsize = (20,12))
    plt.subplot(2,1,1)
    plt.plot(y_train, label = 'train')
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.subplot(2,1,2)
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.show()

### **(3) 데이터 불러오기**

In [6]:
oil_price = pd.read_csv(path + 'train/oil_price_train.csv')
orders = pd.read_csv(path + 'train/orders_train.csv')
sales = pd.read_csv(path + 'train/sales_train.csv')
products = pd.read_csv(path + 'train/products.csv')
stores = pd.read_csv(path + 'train/stores.csv')

sales['Date'] = pd.to_datetime(sales['Date'] )
oil_price['Date'] = pd.to_datetime(oil_price['Date'] )
orders['Date'] = pd.to_datetime(orders['Date'] )

# **2.데이터셋 구성하기**

* 세부 요구사항
    * 상품코드 별로 각각 데이터셋을 구성해야 합니다.
    * 도출된 변수들을 포함하는 데이터셋을 구성합니다.


In [9]:
def data_compose(id):
    df = sales.loc[(sales['Store_ID'] == 44) & (sales['Product_ID'] == id)]
    df = df.merge(orders, how='left', on=['Date', 'Store_ID'])
    df = df.drop(['Store_ID', 'Product_ID'], axis = 1)

    df['CustomerCount'] = df['CustomerCount'].fillna(0)

    df['Qty_RM7'] = df['Qty'].rolling(7, min_periods=1).mean()

    df['Qty_RM14'] = df['Qty'].rolling(14, min_periods=1).mean()

    df['Customer_RM7'] = df['CustomerCount'].rolling(7, min_periods=1).mean()

    df['Customer_RM14'] = df['CustomerCount'].rolling(14, min_periods=1).mean()

    # 이틀 후(예측일)의 1주일 전 Qty와 CC
    df['Qty_before_5d'] = df['Qty'].shift(5)
    df['Qty_before_5d'] = df['Qty_before_5d'].fillna(df['Qty'])

    df['Customer_before_5d'] = df['CustomerCount'].shift(5)
    df['Customer_before_5d'] = df['Customer_before_5d'].fillna(df['CustomerCount'])

    df['Qty_before_7d'] = df['Qty'].shift(7)
    df['Qty_before_7d'] = df['Qty_before_7d'].fillna(df['Qty'])

    df['Customer_before_7d'] = df['CustomerCount'].shift(7)
    df['Customer_before_7d'] = df['Customer_before_7d'].fillna(df['CustomerCount'])

    df['Weekend'] = df['Date'].dt.dayofweek.apply(lambda x: 1 if x in [5, 6] else 0)
    df['Season'] = df['Date'].dt.month.map({1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Winter'})

    df = pd.get_dummies(df, columns=['Season'], drop_first=True)
    df = df.replace({False: 0, True: 1})

    df['Holiday'] = df['Date'].apply(lambda x: 1 if x.month == 1 and x.day == 1 else 0)

    df['y'] = df['Qty'].shift(-2)
    # df['y'] = df['y'].fillna(method='bfill')

    df = df.iloc[:-2]

    df = df.set_index('Date')

    return df

## **(1) 상품 : 3 - Beverage**

In [10]:
df_3 = data_compose(3)

In [11]:
df_3.head(10)

,Qty,CustomerCount,Qty_RM7,Qty_RM14,Customer_RM7,Customer_RM14,Qty_before_5d,Customer_before_5d,Qty_before_7d,Customer_before_7d,Weekend,Season_Spring,Season_Summer,Season_Winter,Holiday,y
Date,,,,,,,,,,,,,,,,
2014-01-01,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0,0,0,1,1,8647.0
2014-01-02,9853.0,4422.0,4926.500000,4926.500000,2211.000000,2211.000000,9853.0,4422.0,9853.0,4422.0,0,0,0,1,0,15153.0
2014-01-03,8647.0,4167.0,6166.666667,6166.666667,2863.000000,2863.000000,8647.0,4167.0,8647.0,4167.0,0,0,0,1,0,15606.0
2014-01-04,15153.0,5341.0,8413.250000,8413.250000,3482.500000,3482.500000,15153.0,5341.0,15153.0,5341.0,1,0,0,1,0,7900.0
2014-01-05,15606.0,5123.0,9851.800000,9851.800000,3810.600000,3810.600000,15606.0,5123.0,15606.0,5123.0,1,0,0,1,0,7188.0
2014-01-06,7900.0,3917.0,9526.500000,9526.500000,3828.333333,3828.333333,0.0,0.0,7900.0,3917.0,0,0,0,1,0,8800.0
2014-01-07,7188.0,3663.0,9192.428571,9192.428571,3804.714286,3804.714286,9853.0,4422.0,7188.0,3663.0,0,0,0,1,0,6606.0
2014-01-08,8800.0,4254.0,10449.571429,9143.375000,4412.428571,3860.875000,8647.0,4167.0,0.0,0.0,0,0,0,1,0,7051.0
2014-01-09,6606.0,3723.0,9985.714286,8861.444444,4312.571429,3845.555556,15153.0,5341.0,9853.0,4422.0,0,0,0,1,0,12599.0


In [ ]:
df_3.tail(10)

In [ ]:
scaler = MinMaxScaler()
df_3_scaled = scaler.fit_transform(df_3)

df_3 = pd.DataFrame(df_3_scaled, columns=df_3.columns)

## **(2) 상품 : 12 - Milk**

In [ ]:
df_12 = data_compose(12)

In [ ]:
df_12.head(10)

In [ ]:
df_12.tail(10)

In [ ]:
df_12_scaled = scaler.fit_transform(df_12)

df_12 = pd.DataFrame(df_12_scaled, columns=df_12.columns)

## **(3) 상품 : 42 - Agricultural products**

In [ ]:
df_42 = data_compose(42)

In [ ]:
df_42.head(10)

In [ ]:
df_42.tail(10)

In [ ]:
df_42_scaled = scaler.fit_transform(df_42)

df_42 = pd.DataFrame(df_42_scaled, columns=df_42.columns)

# **3.Baseline Model**

* **세부 요구사항**
    * 데이터 준비
        * 스케일링
            * 교과서 적으로는 데이터 분할 이후 스케일링하는게 맞지만, 데이터 건수가 충분히 크다면, 스케일링을 먼저 하기도 합니다.
            * 여기서는 먼저 스케일링을 한 후 데이터 분할을 합니다.
        * 데이터 분할
            * 검증셋 구성 : 학습용 데이터에서 최근 60일을 검증셋으로 사용.

    * 모델링
        * Dense Layer 1개(output layer)만으로 모델을 생성합니다.
        * 모델의 검증 성능을 평가하고 기록합니다.
        * 평가 지표 : RMSE, MAE, MAPE, R2 Score


In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mcp = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

## **(1) 상품 : 3 - Beverage**

### **1) 데이터 준비**

In [ ]:
X = df_3.drop(['y'], axis = 1)
y = df_3['y']

X_train, X_valid, y_train, y_valid = X.iloc[:-60], X.iloc[-60:], y.iloc[:-60], y.iloc[-60:]

### **2) 모델링**

In [ ]:
clear_session()

model = Sequential()
model.add(Dense(1, input_shape=(X_train.shape[1],)))
model.summary()

In [ ]:
model.compile(loss = 'mse', optimizer = 'adam')
history = model.fit(X_train, y_train, epochs = 500, validation_data = (X_valid, y_valid), callbacks=[es, mcp])

In [ ]:
dl_history_plot(history.history)

In [ ]:
y_pred = model.predict(X_valid)
print('RMSE : ', np.sqrt(mean_squared_error(y_valid, y_pred)))
print('MAE : ', mean_absolute_error(y_valid, y_pred))
print('MAPE : ', mean_absolute_percentage_error(y_valid, y_pred))
print('R2 Score : ', r2_score(y_valid, y_pred))

In [ ]:
plot_model_result(y_train, y_valid, y_pred)

## **(2) 상품 : 12 - Milk**

### **1) 데이터 준비**

In [ ]:
X = df_12.drop(['y'], axis = 1)
y = df_12['y']

X_train, X_valid, y_train, y_valid = X.iloc[:-60], X.iloc[-60:], y.iloc[:-60], y.iloc[-60:]

### **2) 모델링**

In [ ]:
clear_session()

model = Sequential()
model.add(Dense(1, input_shape=(X_train.shape[1],)))
model.summary()

In [ ]:
model.compile(loss = 'mse', optimizer = 'adam')
history = model.fit(X_train, y_train, epochs = 500, validation_data = (X_valid, y_valid), callbacks=[es, mcp])

In [ ]:
dl_history_plot(history.history)

In [ ]:
y_pred = model.predict(X_valid)
print('RMSE : ', np.sqrt(mean_squared_error(y_valid, y_pred)))
print('MAE : ', mean_absolute_error(y_valid, y_pred))
print('MAPE : ', mean_absolute_percentage_error(y_valid, y_pred))
print('R2 Score : ', r2_score(y_valid, y_pred))

In [ ]:
plot_model_result(y_train, y_valid, y_pred)

## **(3) 상품 : 42 - Agricultural products**

### **1) 데이터 준비**

In [ ]:
X = df_42.drop(['y', 'Holiday'], axis = 1)
y = df_42['y']

X_train, X_valid, y_train, y_valid = X.iloc[:-60], X.iloc[-60:], y.iloc[:-60], y.iloc[-60:]

### **2) 모델링**

In [ ]:
clear_session()

model = Sequential()
model.add(Dense(1, input_shape=(X_train.shape[1],)))
model.summary()

In [ ]:
model.compile(loss = 'mse', optimizer = 'adam')
history = model.fit(X_train, y_train, epochs = 500, validation_data = (X_valid, y_valid), callbacks=[es, mcp])

In [ ]:
dl_history_plot(history.history)

In [ ]:
y_pred = model.predict(X_valid)
print('RMSE : ', np.sqrt(mean_squared_error(y_valid, y_pred)))
print('MAE : ', mean_absolute_error(y_valid, y_pred))
print('MAPE : ', mean_absolute_percentage_error(y_valid, y_pred))
print('R2 Score : ', r2_score(y_valid, y_pred))

In [ ]:
plot_model_result(y_train, y_valid, y_pred)

# **4.LSTM, CNN 초기 모델**

* **세부 요구사항**
    * 3차원 데이터 구조 만들기
        * timestep 수는 적절한 간격으로 한가지 경우만 지정을 합니다.
    * LSTM, CNN 기반 초기 모델 생성
        * 성능 보다는 코드 틀을 작성하는데 집중합시다.
        * 노드 혹은 필터 수와 크기는 초기값으로 적절하게 지정해 봅시다.

In [ ]:
def temporalize(x, y, timesteps):
    output_X = []
    output_y = []
    for i in range(len(x) - timesteps + 1):
        output_X.append(x.iloc[i:i+timesteps])
        output_y.append(y.iloc[i+timesteps-1])
    return np.array(output_X), np.array(output_y)

## **(1) LSTM 초기 모델링**

### **1) 상품 : 3 - Beverage**

* **데이터 준비**

In [ ]:
X = df_3.drop(['y'], axis = 1)
y = df_3['y']

X_train, X_valid, y_train, y_valid = X.iloc[:-60], X.iloc[-60:], y.iloc[:-60], y.iloc[-60:]

In [ ]:
X_train_3d, y_train_3d = temporalize(X_train, y_train, 7)
X_valid_3d, y_valid_3d = temporalize(X_valid, y_valid, 7)

* **모델링**

In [ ]:
clear_session()

model = Sequential()
model.add(LSTM(64, input_shape=(X_train_3d.shape[1], X_train_3d.shape[2]), activation='relu'))
model.add(Dense(1))
model.summary()

In [ ]:
model.compile(loss = 'mse', optimizer = 'adam')
history = model.fit(X_train_3d, y_train_3d, epochs = 50, validation_data = (X_valid_3d, y_valid_3d), callbacks=[es, mcp])

In [ ]:
dl_history_plot(history.history)

In [ ]:
y_pred = model.predict(X_valid_3d)
print('RMSE : ', np.sqrt(mean_squared_error(y_valid_3d, y_pred)))
print('MAE : ', mean_absolute_error(y_valid_3d, y_pred))
print('MAPE : ', mean_absolute_percentage_error(y_valid_3d, y_pred))
print('R2 Score : ', r2_score(y_valid_3d, y_pred))

In [ ]:
plot_model_result(y_train_3d, y_valid_3d, y_pred)

### **2) 상품 : 12 - Milk**

* **데이터 준비**

In [ ]:
X = df_12.drop(['y'], axis = 1)
y = df_12['y']

X_train, X_valid, y_train, y_valid = X.iloc[:-60], X.iloc[-60:], y.iloc[:-60], y.iloc[-60:]

In [ ]:
X_train_3d, y_train_3d = temporalize(X_train, y_train, 7)
X_valid_3d, y_valid_3d = temporalize(X_valid, y_valid, 7)

* **모델링**

In [ ]:
clear_session()

model = Sequential()
model.add(LSTM(64, input_shape=(X_train_3d.shape[1], X_train_3d.shape[2]), activation='relu'))
model.add(Dense(1))
model.summary()

In [ ]:
model.compile(loss = 'mse', optimizer = 'adam')
history = model.fit(X_train_3d, y_train_3d, epochs = 50, validation_data = (X_valid_3d, y_valid_3d), callbacks=[es, mcp])

In [ ]:
dl_history_plot(history.history)

In [ ]:
y_pred = model.predict(X_valid_3d)
print('RMSE : ', np.sqrt(mean_squared_error(y_valid_3d, y_pred)))
print('MAE : ', mean_absolute_error(y_valid_3d, y_pred))
print('MAPE : ', mean_absolute_percentage_error(y_valid_3d, y_pred))
print('R2 Score : ', r2_score(y_valid_3d, y_pred))

In [ ]:
plot_model_result(y_train_3d, y_valid_3d, y_pred)

### **3) 상품 : 42 - Agricultural products**

* **데이터 준비**

In [ ]:
X = df_42.drop(['y', 'Holiday'], axis = 1)
y = df_42['y']

X_train, X_valid, y_train, y_valid = X.iloc[:-60], X.iloc[-60:], y.iloc[:-60], y.iloc[-60:]

In [ ]:
X_train_3d, y_train_3d = temporalize(X_train, y_train, 7)
X_valid_3d, y_valid_3d = temporalize(X_valid, y_valid, 7)

* **모델링**

In [ ]:
clear_session()

model = Sequential()
model.add(LSTM(64, input_shape=(X_train_3d.shape[1], X_train_3d.shape[2]), activation='relu'))
model.add(Dense(1))
model.summary()

In [ ]:
model.compile(loss = 'mse', optimizer = 'adam')
history = model.fit(X_train_3d, y_train_3d, epochs = 50, validation_data = (X_valid_3d, y_valid_3d), callbacks=[es, mcp])

In [ ]:
dl_history_plot(history.history)

In [ ]:
y_pred = model.predict(X_valid_3d)
print('RMSE : ', np.sqrt(mean_squared_error(y_valid_3d, y_pred)))
print('MAE : ', mean_absolute_error(y_valid_3d, y_pred))
print('MAPE : ', mean_absolute_percentage_error(y_valid_3d, y_pred))
print('R2 Score : ', r2_score(y_valid_3d, y_pred))

In [ ]:
plot_model_result(y_train_3d, y_valid_3d, y_pred)

## **(2) CNN 초기 모델링**

### **1) 상품 : 3 - Beverage**

* **데이터 준비**

In [ ]:
X = df_3.drop(['y'], axis = 1)
y = df_3['y']

X_train, X_valid, y_train, y_valid = X.iloc[:-60], X.iloc[-60:], y.iloc[:-60], y.iloc[-60:]

In [ ]:
X_train_3d, y_train_3d = temporalize(X_train, y_train, 7)
X_valid_3d, y_valid_3d = temporalize(X_valid, y_valid, 7)

* **모델링**

In [ ]:
clear_session()

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model.add(Flatten())
model.add(Dense(1))
model.summary()

In [ ]:
model.compile(loss = 'mse', optimizer = 'adam')
history = model.fit(X_train_3d, y_train_3d, epochs = 50, batch_size = 1, validation_data = (X_valid_3d, y_valid_3d), callbacks=[es, mcp])

In [ ]:
dl_history_plot(history.history)

In [ ]:
y_pred = model.predict(X_valid_3d)
print('RMSE : ', np.sqrt(mean_squared_error(y_valid_3d, y_pred)))
print('MAE : ', mean_absolute_error(y_valid_3d, y_pred))
print('MAPE : ', mean_absolute_percentage_error(y_valid_3d, y_pred))
print('R2 Score : ', r2_score(y_valid_3d, y_pred))

In [ ]:
plot_model_result(y_train_3d, y_valid_3d, y_pred)

### **2) 상품 : 12 - Milk**

* **데이터 준비**

In [ ]:
X = df_12.drop(['y'], axis = 1)
y = df_12['y']

X_train, X_valid, y_train, y_valid = X.iloc[:-60], X.iloc[-60:], y.iloc[:-60], y.iloc[-60:]

In [ ]:
X_train_3d, y_train_3d = temporalize(X_train, y_train, 7)
X_valid_3d, y_valid_3d = temporalize(X_valid, y_valid, 7)

* **모델링**

In [ ]:
clear_session()

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model.add(Flatten())
model.add(Dense(1))
model.summary()

In [ ]:
model.compile(loss = 'mse', optimizer = 'adam')
history = model.fit(X_train_3d, y_train_3d, epochs = 50, batch_size = 1, validation_data = (X_valid_3d, y_valid_3d), callbacks=[es, mcp])

In [ ]:
dl_history_plot(history.history)

In [ ]:
y_pred = model.predict(X_valid_3d)
print('RMSE : ', np.sqrt(mean_squared_error(y_valid_3d, y_pred)))
print('MAE : ', mean_absolute_error(y_valid_3d, y_pred))
print('MAPE : ', mean_absolute_percentage_error(y_valid_3d, y_pred))
print('R2 Score : ', r2_score(y_valid_3d, y_pred))

In [ ]:
plot_model_result(y_train_3d, y_valid_3d, y_pred)

### **3) 상품 : 42 - Agricultural products**

* **데이터 준비**

In [ ]:
X = df_42.drop(['y', 'Holiday'], axis = 1)
y = df_42['y']

X_train, X_valid, y_train, y_valid = X.iloc[:-60], X.iloc[-60:], y.iloc[:-60], y.iloc[-60:]

In [ ]:
X_train_3d, y_train_3d = temporalize(X_train, y_train, 7)
X_valid_3d, y_valid_3d = temporalize(X_valid, y_valid, 7)

* **모델링**

In [ ]:
clear_session()

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model.add(Flatten())
model.add(Dense(1))
model.summary()

In [ ]:
model.compile(loss = 'mse', optimizer = 'adam')
history = model.fit(X_train_3d, y_train_3d, epochs = 50, batch_size = 1, validation_data = (X_valid_3d, y_valid_3d), callbacks=[es, mcp])

In [ ]:
dl_history_plot(history.history)

In [ ]:
y_pred = model.predict(X_valid_3d)
print('RMSE : ', np.sqrt(mean_squared_error(y_valid_3d, y_pred)))
print('MAE : ', mean_absolute_error(y_valid_3d, y_pred))
print('MAPE : ', mean_absolute_percentage_error(y_valid_3d, y_pred))
print('R2 Score : ', r2_score(y_valid_3d, y_pred))

In [ ]:
plot_model_result(y_train_3d, y_valid_3d, y_pred)

# **5.데이터 저장**

* **세부 요구사항**
    * 상품코드별 데이터셋을 저장하시오.
    * joblib.dump를 이용하시오.
    * 저장할 파일의 확장자는 보통 .pkl 입니다.

In [ ]:
joblib.dump(df_3, path+'train_data_3.pkl')

In [ ]:
joblib.dump(df_12, path+'train_data_12.pkl')

In [ ]:
joblib.dump(df_42, path+'train_data_42.pkl')